#### Code A


In [ ]:
import os
from ultralytics import YOLO
import shutil


# Path to the folder you want to delete
folder_path = "checkpoints/train"

# Check if the folder exists
if os.path.exists(folder_path):
    # Delete the folder and all its contents
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")
else:
    print(f"Folder '{folder_path}' does not exist.")

<br>

#### Code B

In [ ]:
# 하이퍼파라미터 설정
EPOCH = 100        # 총 학습 에포크 수
BATCH_SIZE = 16    # 배치 크기

<br>

#### Code C

In [ ]:
# Load the locally downloaded YOLOv8n model
model = YOLO("yolov8n.pt")

<br>

#### Code D

In [ ]:
from ultralytics import YOLO
from ultralytics.utils import LOGGER
import logging
import torch
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as display

LOGGER.propagate = False

# Store loss values
epochs = []
box_losses = []
cls_losses = []
dfl_losses = []

# Function to log loss and update the graph
def loss_logger(trainer):
    if hasattr(trainer, "tloss") and trainer.tloss is not None:
        try:
            # Convert tensor loss to list (if applicable)
            if isinstance(trainer.tloss, torch.Tensor):
                loss_values = trainer.tloss.tolist()
            else:
                loss_values = [trainer.tloss]  # If it's a float, wrap it in a list

            # Extract individual loss values (handle case where only one value is present)
            if len(loss_values) >= 3:
                box_loss, cls_loss, dfl_loss = loss_values[:3]
            elif len(loss_values) == 2:
                box_loss, cls_loss = loss_values[:2]
                dfl_loss = 0
            else:
                box_loss = loss_values[0]
                cls_loss, dfl_loss = 0, 0  # Default missing values to 0

            display.clear_output(wait=True)

            # Store losses for plotting
            epochs.append(trainer.epoch + 1)
            box_losses.append(box_loss)
            cls_losses.append(cls_loss)
            dfl_losses.append(dfl_loss)

            print(f'[에폭 {trainer.epoch + 1}] 박스 손실: {box_loss:.3f} | '
      f'분류 손실: {cls_loss:.3f} | DFL 손실: {dfl_loss:.3f}')

            # Plot losses dynamically
            plt.figure(figsize=(10, 5))
            plt.plot(epochs, box_losses, marker='o', linestyle='-', label='Box Loss', color='b', alpha=0.7)
            plt.plot(epochs, cls_losses, marker='s', linestyle='-', label='Cls Loss', color='g', alpha=0.7)
            plt.plot(epochs, dfl_losses, marker='^', linestyle='-', label='DFL Loss', color='r', alpha=0.7)

            # Graph styling
            plt.title('YOLO Training Loss per Epoch')
            plt.xlabel('Epoch')
            plt.ylabel('Loss Value')
            plt.legend()
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Loss logging error: {e}")

# Register the callback
model.add_callback("on_train_epoch_end", loss_logger)  # Changed to on_train_epoch_end for better loss tracking

# Train the model
results = model.train(
    data="data2.yaml",
    epochs=100,
    batch=16,
    imgsz=640,
    project="checkpoints",
    save=True,
    exist_ok=True,
    save_period=10,
    plots=False
)

<br></br>